In [4]:
!pip install geetools 
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74 kB 2.7 MB/s 
     |████████████████████████████████| 46 kB 3.9 MB/s 
  Created wheel for geetools: filename=geetools-0.6.14-py3-none-any.whl size=92107 sha256=628f22d14075887ff772d875dd17e530c24b1455cb318f041d1e6f462b314fc0
  Stored in directory: /root/.cache/pip/wheels/5c/55/29/0a09dcf6b39bba8c890e40171cac3e3607a5c4b354b7e9447c
Successfully built geetools


In [130]:
import ee
import csv
import time
from geetools import batch
from tqdm import tqdm
from argparse import ArgumentParser
import requests
import matplotlib.pyplot as plt
import geemap
import folium
import math

In [9]:
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=vxUWNlj6paC7YYsk5gsFfTosGbqcYpXx6KU2h_68Q2w&tc=m2O3GOzf99xxjDhPxEjtTI-BIJU_k0Q1_ISn3zPFGsQ&cc=qHF2oyREoRmridosMR8lRAoyH0xbO-pDiWmM0FWAoPw

The authorization workflow will generate a code, which you should paste in the box below. 
Enter verification code: 4/1AX4XfWi38okSGCWHZBry9OvMMm3M5tud8nrbjs1xSmDtQOglTh7JGk66CQw

Successfully saved authorization token.


In [10]:
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 4000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}

# Create a folium map object.
my_map = folium.Map(location=[20, 0], zoom_start=3)

# Add the elevation model to the map object.
my_map.add_ee_layer(dem.updateMask(dem.gt(0)), vis_params, 'DEM')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)

In [12]:
##might have to come up with different formula 
## concern 1: does this result in the desired pixel dimesions of image
## concern 2: should we not use radial distance instead? 
# def getSquareImageCorners(lat, lon, height, width):
#     offset = 0.2
#     xMin = lon - offset
#     xMax = lon + offset  
#     yMin = lat - offset
#     yMax = lat + offset
#     return xMin, xMax, yMin, yMax 

In [120]:
## find a bounding box
# resolution = input()
resLandsat = 30
def boundingBox(lat, lon, size, res):

  earth_radius = 6371000
  angular_distance = math.degrees(0.5 * ((size * res) / earth_radius))
  osLat = angular_distance
  osLon = angular_distance / math.cos(math.radians(lat))
  xMin = lon - osLon
  xMax = lon + osLon
  yMin = lat - osLat
  yMax = lat + osLat
  return xMin, xMax, yMin, yMax

In [121]:
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_TOA")

In [122]:
start_date = '2017-01-01'
end_date = '2018-01-01'

In [ ]:
# RGB_VIZ = {'bands':['B4', 'B3', 'B2'], 'min':0, 'max':0.4}

# res = 30 

# crs = 'EPSG:3857'
# crs2 = 'EPSG:4326'
# fileFormat = 'GeoTIFF'
# cloud_percent = 20

# #   with open(filepath) as coords_file:
# #     coords = csv.reader(coords_file)
# #     for coord in coords:
# #         lat = coord[0]
# #         lon = coord[1]


# lat = 35.994034
# lon = -78.898621
# description = 'test_image_0'
# start='2017-01-01'
# end='2020-01-01'
# height = 512

# xMin, xMax, yMin, yMax = boundingBox(lat, lon, height, res)
# geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])

# filtered = landsat.filterDate(start, end).filterBounds(geometry)
# image = filtered.median().clip(geometry).select(['B2', 'B3', 'B4', 'B5'])
# #image = image.visualize(RGB_VIZ)

# map = folium.Map(location=[lat, lon], zoom_start=10)
# #map.add_ee_layer(image, RGB_VIZ, 'rgb')
# display(map)


In [133]:
def export_location(filepath='coordinates.csv',
                    listOfCoords=[(35.994034, -78.898621)], 
                    dataset=landsat, 
                    start='2017-01-01', end='2020-01-01', 
                    bands=['B2', 'B3', 'B4', 'B5'], 
                    height=512, 
                    width=512):

  ''' TODO:

          add default values
          add number of images to get param + interval between images? --> multi-temporal views (default=1)
          change listOfCoords to file that has all coords
          add support for pan-sharpening
  '''

  RGB_VIZ = {'min':0, 'max':0.4, 'bands':['B4', 'B3', 'B2']}

  res = 30 if dataset == landsat else 10
  
  crs = 'EPSG:3857'
  crs2 = 'EPSG:4326'
  fileFormat = 'GeoTIFF'
  cloud_percent = 20

#   with open(filepath) as coords_file:
#     coords = csv.reader(coords_file)
#     for coord in coords:
#         lat = coord[0]
#         lon = coord[1]

    # Define the visualization parameters.
  image_viz_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.5}


  for idx, coord in enumerate(listOfCoords):
    lat = coord[0]
    lon = coord[1]
    description = f'image_{idx}'

    xMin, xMax, yMin, yMax = boundingBox(lat, lon, height, res)
    geometry = ee.Geometry.Rectangle([[xMin, yMin],[xMax, yMax]])

    RGB = ['B4', 'B3', 'B2'] if dataset == landsat else ['R', 'G', 'B']
    filtered = dataset.filterDate(start, end).filterBounds(geometry).select(RGB)
    image = ee.Image(filtered.median()).clip(geometry)
    ##Landsat specific!! should change params depending on dataset
    img = image.visualize(bands = RGB, min = 0, max = 0.5)
    
    ## OPTIONAL: just to view the map, to comment out when exporting more than 1 image
    map = folium.Map(location=[lat, lon], zoom_start=10)
    map.add_ee_layer(image, image_viz_params, 'rgb')
    display(map)



## later add if clause to run this if we want links

    # url = image.getDownloadUrl({
    # 'description':description,
    # 'region':geometry,
    # 'fileNamePrefix':description,
    # 'crs':crs,
    # 'fileFormat':fileFormat,
    # 'bands': ['B4', 'B3', 'B2'],
    # 'region': geometry,
    # 'format': 'GEO_TIFF',
    # #'scale': res,
    # 'dimensions': [height, width]
    # })

    ## when exported using scale and region, images are 634x514, whereas specifiying dimesions forces the image to be of requested dims
    ## exporting images using getDownloadUrl reuslts in ~700 kb files, whereas exporting to Drive results in 32+ Mb images

    # response = requests.get(url)
    # with open('test.tif', 'wb') as fd:
    #   fd.write(response.content)


    local_save = geemap.ee_export_image(img, 
                          filename=description+'.tif', 
                          scale=res, 
                          region=geometry,
                          file_per_band=False)
    

    export_start_time = time.time()
    task = ee.batch.Export.image.toDrive(image=img,
                                     description=description,
                                     scale=res, 
                                     region=geometry,
                                     fileNamePrefix=description,
                                     crs=crs,
                                     fileFormat=fileFormat,
                                     folder='June6 test')
    task.start()

    return image, task, export_start_time


In [134]:
myimg, mytask, export_start_time = export_location(filepath='coordinates.csv',
                    listOfCoords=[(35.994034, -78.898621)], 
                    dataset=landsat, 
                    start=start_date, 
                    end=end_date, 
                    bands=['B2', 'B3', 'B4', 'B5'], 
                    height=512, 
                    width=512)

Generating URL ...
Please wait ...
Data downloaded to /content/image_0.tif


In [135]:
mytask.status()

{'creation_timestamp_ms': 1654552074736,
 'description': 'image_0',
 'id': 'F6UVGVLOJUNXJYJ5U7HUFB4C',
 'name': 'projects/earthengine-legacy/operations/F6UVGVLOJUNXJYJ5U7HUFB4C',
 'start_timestamp_ms': 0,
 'state': 'READY',
 'task_type': 'EXPORT_IMAGE',
 'update_timestamp_ms': 1654552074736}

In [139]:
state = mytask.status()['state']
print(state)
while state != 'COMPLETED':
  previous = mytask.status()['state']
  if mytask.status()['state'] != previous:
    print(state)
    previous = state
  if state == 'FAILED':
    break
print('export complete!')
export_finish_time = time.time()
duration = export_finish_time - export_start_time
print(f'exporting took: {duration} s')

COMPLETED
export complete!
exporting took: 184.2360281944275 s
